# Lab 2

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import seaborn
import re
import math
import time
import tensorflow as tf
import sys
import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as pyplot
import scipy.stats as stats
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow import keras
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Dense, Input
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Layer, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model, Sequential
from keras.utils.vis_utils import plot_model

In [3]:
print("Tensorflow version: ", tf.__version__)
print("Keras Version: ", keras.__version__)
print("System Version: ", sys.version)

Tensorflow version:  2.9.2
Keras Version:  2.9.0
System Version:  3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]


# Load ConceptNet Embedding

In [10]:
EMBEDDING_LOCATIONS = {
    "NUMBER_BATCH": "/notebooks/data/embeddings/numberBatch/numberbatch-en-17.04b.txt"
}

In [8]:
# *************************************************************************************************************************************************
# Robyn Speer (2017) How to make a racist AI without really trying [source code]. https://blog.conceptnet.io/posts/2017/how-to-make-a-racist-ai-without-really-trying/
# *************************************************************************************************************************************************
def load_embeddings(filename):
    """
    Load a DataFrame from the generalized text format used by word2vec, GloVe,
    fastText, and ConceptNet Numberbatch. The main point where they differ is
    whether there is an initial line with the dimensions of the matrix.
    """
    labels = []
    rows = []
    with open(filename, encoding='utf-8') as infile:
        for i, line in enumerate(infile):
            items = line.rstrip().split(' ')
            if len(items) == 2:
                # This is a header row giving the shape of the matrix
                continue
            labels.append(items[0])
            values = np.array([float(x) for x in items[1:]], 'f')
            rows.append(values)
    
    arr = np.vstack(rows)
    return pd.DataFrame(arr, index=labels, dtype='f')

In [ ]:
%%time 
number_batch_embedding = load_embeddings(EMBEDDING_LOCATIONS["NUMBER_BATCH"])
number_batch_embedding.shape

# Load Dataset

## IMDB